# Follow-up filtering
This primarily exists such that it is unecessary to run the full extraction process again while I find additional items to filter during the cleaning process

In [1]:
import json
from pathlib import Path
import ujson
from dutchanalyzer.config import *
from dutchanalyzer.utilities.utils import *
from dutchanalyzer.utilities.json_utils import *
from dutchanalyzer.utilities.replacement_utils import *
from pathlib import Path
from dotenv import load_dotenv
import datetime
from pprint import pprint
from tqdm import tqdm

In [13]:
from dutchanalyzer.utilities.pandas_utils import *

In [2]:
today = datetime.date.today().__format__("%d-%m-%y")
current_save_folder = Path(INTERIM_DATA_DIR, 'cleaning', 'wikt', str(today))
folders = {'en': ['EEF', 'ENF'], 'nl':['NEF', 'NNF']}


for k, v in folders.items():
    for f in v:
        Path.mkdir(Path(current_save_folder, k, f), parents=True, exist_ok=True)

In [3]:
NNR_file = Path(NNR_DIR, 'NNR.jsonl')
NER_file = Path(NER_DIR, 'NER.jsonl')
EER_file = Path(EER_DIR, 'EER.jsonl')
ENR_file = Path(ENR_DIR, 'ENR.jsonl')

In [7]:
def filter_obj(obj):
    obj.pop('wikipedia', '')
    

In [8]:
def process_obj(in_file, entries_out_file, wl_code, definitions_out_file=None, batch_size=1000, break_point=-1):
    batch = []
    entries_batch = []
    error_lines = []
    with open(in_file, 'r', encoding='utf-8') as f:
        with open(entries_out_file, 'w+', encoding='utf-8') as out:
            
            for i, line in tqdm(enumerate(f)):
                if break_point > 0:
                    if i > break_point:
                        display(entries_batch)
                        break
                if line:
                    try:
                        obj = json.loads(line)
                        obj = sort_standardize_entry(obj)
                        if obj:
                            obj['wl_code'] = wl_code
                            filter_obj(obj)
                            entries_batch.append(obj)

                            if len(entries_batch) > batch_size:
                                for entry in entries_batch:
                                    json.dump(entry, out, ensure_ascii=False)
                                    out.write('\n')
                                entries_batch = []
                            if definitions_out_file:
                                word_entry = extract_words_senses(obj)
                                batch.append(word_entry)
                                if len(batch) > batch_size:
                                    with open(definitions_out_file, 'a+', encoding='utf-8') as def_out: 
                                        for entry in batch:
                                            json.dump(entry, def_out, ensure_ascii=False)
                                            def_out.write('\n')
                                        batch = []
                    except Exception as e:
                        error_lines.append((i, obj))
                        display(line)
                        print("Error on line: ", i, " Error: ", e)
                        break
            if entries_batch:
                for entry in entries_batch:
                    json.dump(entry, out, ensure_ascii=False)
                    out.write('\n')  
            if batch and definitions_out_file:
                with open(definitions_out_file, 'a+', encoding='utf-8') as def_out: 
                    for entry in batch:
                        json.dump(entry, def_out, ensure_ascii=False)
                        def_out.write('\n')
    return entries_batch, batch, error_lines

In [44]:
file = EEF_FILE
temp_out = Path(current_save_folder, 'en', 'EEF', 'EEF_temp.jsonl')
defs_file = Path(current_save_folder, 'en', 'EEF_definitions.jsonl')
wl_code = 'EEF'
entries_batch, batch, error_lines = process_obj(file, temp_out, wl_code, defs_file, 1000, 100)

0it [00:00, ?it/s]

[{'word': 'dictionary',
  'pos': 'noun',
  'lang_code': 'en',
  'lang': 'english',
  'standard_lang': 'english',
  'categories': ['English countable nouns',
   'English entries with etymology trees',
   'English entries with incorrect language header',
   'English lemmas',
   'English nouns',
   'English terms derived from Latin',
   'English terms derived from Medieval Latin',
   'English terms derived from Middle English',
   'English terms derived from Proto-Indo-European',
   'English terms derived from Proto-Italic',
   'English terms derived from the Proto-Indo-European root *deyḱ-',
   'English terms inherited from Middle English',
   'English terms suffixed with -ary',
   'English verbs',
   'Entries with translation boxes',
   'Pages with 1 entry',
   'Pages with entries',
   'Pages with etymology trees',
   'Rhymes:English/ɪkʃənɛəɹi',
   'Rhymes:English/ɪkʃənɛəɹi/4 syllables',
   'Terms with Dutch Low Saxon translations',
   'Terms with Dutch translations',
   'en:Dictionarie

101it [00:00, 439.70it/s]


In [45]:
df = pd.DataFrame(entries_batch)

In [46]:
def sort_df_columns(df, start_cols=['word', 'pos', 'lang_code', 'standard_lang', 'senses'], end_cols=['translations', 'lang', 'wl_code'], groups=['forms', 'etymology', 'nyms', 'categories']):
    df_cols = df.columns
    start = [c for c in start_cols if c in df_cols]
    end = [c for c in end_cols if c in df_cols]
    protected_cols = start 
    
    forms_grouping = ['form_of','forms', 'alt_of', 'inflection_templates', 'derived']
    etymology_grouping = ['etymology_templates', 'etymology_text', 'etymology_tree']
    nyms_grouping = ['synonyms', 'antonyms', 'hypernyms','hyponyms', 'troponyms', 'holonyms', 'meronyms']
    categories_grouping = ['categories', 'links', 'related', 'topics']

    if groups:
        for group in groups:
            if 'forms' == group:
                forms_grouping = [c for c in forms_grouping if c in df_cols]
                protected_cols += forms_grouping
            if 'etymology' == group:
                etymology_grouping = [c for c in etymology_grouping if c in df_cols]
                protected_cols += etymology_grouping

            if 'nyms' == group:
                nyms_grouping = [c for c in nyms_grouping if c in df_cols]
                protected_cols += nyms_grouping

            if 'categories' == group:
                categories_grouping = [c for c in categories_grouping if c in df_cols]
                protected_cols += categories_grouping

    protected_cols = [c for c in protected_cols if c in df_cols]
    unprotected_cols = [c for c in df_cols if c not in protected_cols and c not in end]

    unprotected_cols.sort()
    new_cols = protected_cols + unprotected_cols + end
    df = df.loc[:, new_cols]
    return df

In [47]:
df = sort_df_columns(df)

In [48]:
def summarize_df(df: pd.DataFrame, name: str = None, num_top_values=10) -> pd.DataFrame:
    """
    Summarize a DataFrame similar to df.info(), but with:
      - shape
      - non-null count
      - unique count
      - top 10 value counts (as string)
    """
    summary_data = []

    for i, col in enumerate(df.columns):
        non_null = df[col].notna().sum()
        try: 
            unique_vals = df[col].nunique(dropna=True)
        except:
            unique_vals = 'List'
        top_vals = df[col].value_counts(dropna=False).head(num_top_values)
        top_vals_str = ", ".join(f"{idx} : {val}" for idx, val in top_vals.items())

        summary_data.append({
            "df_name": name,
            'col_num': i,
            "column": col,
            "dtype": str(df[col].dtype),
            "non_null": non_null,
            "unique": unique_vals,
            "top_vals": top_vals_str,
            "sh_rows": df.shape[0],
            "sh_cols": df.shape[1],
        })
    summary_df = pd.DataFrame(summary_data)
  
    return summary_df

In [49]:
pd.set_option('display.max_colwidth', 3) 
summarize_df(df, 'EEF', 5)

df_name  col_num               column    dtype  non_null unique  \
0   EEF      0       word                 object   101       37      
1   EEF      1       pos                  object   101        8      
2   EEF      2       lang_code            object   101        1      
3   EEF      3       standard_lang        object   101        1      
4   EEF      4       senses               object   101       List    
5   EEF      5       forms                object   81        List    
6   EEF      6       derived              object   47        List    
7   EEF      7       etymology_templates  object   87        List    
8   EEF      8       etymology_text       object   92        56      
9   EEF      9       synonyms             object   18        List    
10  EEF     10       antonyms             object    3        List    
11  EEF     11       hypernyms            object   14        List    
12  EEF     12       hyponyms             object   15        List    
13  EEF     13       holonyms             object    4        List    
14  EEF     14       meronyms             object    9        List    
15  EEF     15       categories           object   101       List    
16  EEF     16       related              object   41        List    
17  EEF     17       coordinate_terms     object    3        List    
18  EEF     18       descendants          object    9        List    
19  EEF     19       etymology_number     float64  64         6      
20  EEF     20       head_templates       object   101       List    
21  EEF     21       hyphenations         object   26        List    
22  EEF     22       instances            object    1        List    
23  EEF     23       sounds               object   100       List    
24  EEF     24       translations         object   45        List    
25  EEF     25       lang                 object   101        1      
26  EEF     26       wl_code              object   101        1      

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [ ]:
df.info

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   word                 101 non-null    object 
 1   pos                  101 non-null    object 
 2   lang_code            101 non-null    object 
 3   standard_lang        101 non-null    object 
 4   senses               101 non-null    object 
 5   forms                81 non-null     object 
 6   derived              47 non-null     object 
 7   etymology_templates  87 non-null     object 
 8   etymology_text       92 non-null     object 
 9   synonyms             18 non-null     object 
 10  antonyms             3 non-null      object 
 11  hypernyms            14 non-null     object 
 12  hyponyms             15 non-null     object 
 13  holonyms             4 non-null      object 
 14  meronyms             9 non-null      object 
 15  categories           101 non-null    obj

In [ ]:
def drop_df_column_groups(df, groups=['forms', 'etymology', 'nyms', 'categories'], individual_cols=['wl_code', 'lang']):
    df_cols = df.columns
    ind_cols = [c for c in individual_cols if c in df_cols]
    cols_to_drop = ind_cols
    
    forms_grouping = ['form_of','forms', 'alt_of', 'inflection_templates', 'derived']
    etymology_grouping = ['etymology_templates', 'etymology_text', 'etymology_tree']
    nyms_grouping = ['synonyms', 'antonyms', 'hypernyms','hyponyms', 'troponyms', 'holonyms', 'meronyms']
    categories_grouping = ['categories', 'links', 'related', 'topics']

    if groups:
        for group in groups:
            if 'forms' == group:
                forms_grouping = [c for c in forms_grouping if c in df_cols]
                protected_cols += forms_grouping
            if 'etymology' == group:
                etymology_grouping = [c for c in etymology_grouping if c in df_cols]
                protected_cols += etymology_grouping

            if 'nyms' == group:
                nyms_grouping = [c for c in nyms_grouping if c in df_cols]
                protected_cols += nyms_grouping

            if 'categories' == group:
                categories_grouping = [c for c in categories_grouping if c in df_cols]
                protected_cols += categories_grouping

    protected_cols = [c for c in protected_cols if c in df_cols]
    unprotected_cols = [c for c in df_cols if c not in protected_cols and c not in end]

    unprotected_cols.sort()
    new_cols = protected_cols + unprotected_cols + end
    df = df.loc[:, new_cols]
    return df